# I. Importation des bibliothèques

In [1]:
%%capture
!pip install --quiet fuzzywuzzy python-Levenshtein excel-formulas-calculator
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
%%capture
import warnings

warnings.filterwarnings("ignore")
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
import math

import openpyxl as pyxl
from openhexa.sdk import workspace

try:
    from efc.interfaces.iopenpyxl import OpenpyxlInterface
except ImportError or ModuleNotFoundError:
    !pip install --quiet excel-formulas-calculator
    from efc.interfaces.iopenpyxl import OpenpyxlInterface

## 1. Rafraîchissement des bibliothèques

In [3]:
%%capture
# Ajout du chemin d'accès pour l'importation des bibliothèques
os.chdir(Path(workspace.files_path, "Fichier Suivi de Stock/code/pipelines"))

from importlib import reload

import compute_indicators
import export_file_to_google_drive as ggdrive
import generate_stock_tracking_file as gstf

# Importation des réquêtes sql
from compute_indicators.queries import QUERY_ETAT_STOCK_PROGRAMME, QUERY_ETAT_STOCK_PERIPH
from database_operations import process_statut_prod, stock_sync_manager

# Reload modules
reload(compute_indicators)
reload(stock_sync_manager)
reload(gstf)
reload(ggdrive)

# II. Définition des paramètres

## 1. Variables requises pour l'actualisation du fichier de suivi de stock

1. **Mois de création du rapport** : Fourni en tant qu'entrée du pipeline.

2. **Année de conception du rapport**: Fourni en tant qu'entrée du pipeline.

3. **Programme** : Le programme concerné pour lequel le fichier de suivi de stock.

4. **Fichier Suivi de Stock** : Le fichier suivi de stock finalisé après la réunion mensuelle.

In [4]:
month_report, year_report, programme, fp_suivi_stock = (
    "Mars",
    2025,
    "PNLP",
    "Fichier Suivi de Stock PNLP-MARS-2025.xlsx",
)

In [5]:
month_export, date_report = month_report, compute_indicators.utils.format_date(
    month_report, year_report
)

## 2. Test pour s'assurer qu'il y a bien des données déjà présentes dans la base de données pour ce programme en question

L'idée est de vérifier au préalable, avant d'apporter des modifications au programme en question, que des données existent déjà dans la base de données.

In [6]:
stock_sync_manager.initialize_database_connection()

schema_name = "suivi_stock"

Connexion à la base de données établie avec succès


In [7]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select * 
    from {schema_name}.stock_track st 
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    limit 2
    """
)

assert (
    df_.shape[0] != 0
), f"Le mois séléecitonné {date_report} n'a pas de données présente dans la base de données"

del df_

# III.📥Importation des Données
  
L'utilisateur doit veiller à ce que les fichiers respectent le format attendu et soient placés dans les répertoires dédiés avant de procéder au traitement.

## 📌1. Importation du fichier `Fichier Suivi des Stocks`

- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire dédié :  
  **`Fichier Suivi de Stock/data/<programme>/Fichier Suivi de Stock`**
  
- **En cas d'erreur :**  
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que toutes les colonnes requises sont bien renseignées.  
  - Contrôlez que le fichier est bien accessible et non corrompu.

In [8]:
fp_suivi_stock = (
    Path(workspace.files_path)
    / f"Fichier Suivi de Stock/data/{programme}/Fichier Suivi de Stock"
    / Path(fp_suivi_stock).name
)
src_wb = pyxl.load_workbook(fp_suivi_stock)
sheetnames = src_wb.sheetnames

## 📌2. Importation de la feuille `Etat de stock de la NPSP`

In [9]:
sheet_stock_npsp = compute_indicators.utils.check_if_sheet_name_in_file(
    "Etat de stock", sheetnames
)

assert sheet_stock_npsp is not None, print(
    f"La feuille `Etat de stock` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_etat_stock_npsp = pd.read_excel(
    fp_suivi_stock, sheet_name=sheet_stock_npsp, skiprows=4
)

del sheet_stock_npsp

df_etat_stock_npsp = df_etat_stock_npsp.loc[df_etat_stock_npsp["Nouveau code"].notna()]

df_etat_stock_npsp = compute_indicators.file_utils.process_etat_stock_npsp(
    df_etat_stock_npsp, date_report, programme
)

df_etat_stock_npsp.head(2)

,date_report,code_produit,designation,contenance,dmm,traceurs,stock_theorique_bke,stock_theorique_abj,stock_theorique_central,stock_theorique_fin_mois,msd,statut_stock,programme
1,2025-03-01,3010049.0,PARACETAMOL 100 mg comp. BTE/100,BOITE/100,5357.166667,1.0,21379.0,71018.0,0.0,92397.0,17.247363,Surstock,PNLP
2,2025-03-01,3010062.0,PARACETAMOL 250 mg comp BTE/100,BOITE/100,6458.166667,1.0,13877.0,25574.0,0.0,39451.0,6.1087,Bon,PNLP


## 📌3. Importation de la feuille `Stock detaille`

In [10]:
sheet_stock_detaille = compute_indicators.utils.check_if_sheet_name_in_file(
    "Stock detaille", sheetnames
)

assert sheet_stock_detaille is not None, print(
    f"La feuille `Stock detaille` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_stock_detaille = pd.read_excel(fp_suivi_stock, sheet_name=sheet_stock_detaille)

max_date_year = pd.Timestamp.max.year

try:
    df_stock_detaille["Date limite de consommation"] = df_stock_detaille[
        "Date limite de consommation"
    ].apply(
        lambda x: x if x.year < max_date_year else x.replace(year=max_date_year - 1)
    )
except Exception:
    df_stock_detaille["Date limite de consommation"] = df_stock_detaille[
        "Date limite de consommation"
    ].str.strip()

    df_stock_detaille["Date limite de consommation"] = df_stock_detaille[
        "Date limite de consommation"
    ].apply(
        lambda x: x
        if int(x[-4:]) < max_date_year
        else x.replace(x[-4:], str(max_date_year - 1))
    )

df_stock_detaille["Date limite de consommation"] = pd.to_datetime(
    df_stock_detaille["Date limite de consommation"], format="%d/%m/%Y"
)

del sheet_stock_detaille, max_date_year

df_stock_detaille.head(2)

,Code produit,Désignation,Emplacement,Date limite de consommation,Numéro Lot,Sous lot (Programme),Qté \nPhysique,Qté \nlivrable,Unit,Jour restants,Code Couleur
0,4150558,MILDA STANDARD/DELTAMETHRINE,27Q00A00,2099-01-01,NEANT,PNLP,3245,0,UN,26882,GREEN
1,4150558,MILDA STANDARD/DELTAMETHRINE,27Q00A00,2099-01-01,NEANT,PNLP,139500,0,UN,26882,GREEN


## 📌4. Importation de la feuille `Distribution X3`

In [11]:
sheet_distribution_x3 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Distribution X3", sheetnames
)

assert sheet_distribution_x3 is not None, print(
    f"La feuille `Distribution X3` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_distribution = pd.read_excel(fp_suivi_stock, sheet_name=sheet_distribution_x3)

del sheet_distribution_x3

df_distribution.head(2)

,Programme,Date commande,No commande,Date livraison,No livraison,Référence,Client,Raison sociale,Article,Désignation,Unité vente,Qté commandée,Quantité livrée
0,PNLP,NaN,VAL01-PSG-2502-0230,2025-03-03,BL01-PSG-2503-0187,NaN,42400024,HOPITAL GENERAL GUEYO,3050016,ARTESUNATE/AMODIAQUINE 50 / 135 mg ENFANT (1 -...,BTE,1,1
1,PNLP,NaN,VAL01-PSG-2502-0230,2025-03-03,BL01-PSG-2503-0187,NaN,42400024,HOPITAL GENERAL GUEYO,3050062,ARTEMETHER/LUMEFANTRINE 20 / 120 mg (3 - 8 ANS...,BTE,21,21


## 📌5. Importation de la feuille `Receptions`

In [12]:
sheet_reception = compute_indicators.utils.check_if_sheet_name_in_file(
    "Receptions", sheetnames
)

assert sheet_reception is not None, print(
    f"La feuille `Receptions` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_receptions = pd.read_excel(fp_suivi_stock, sheet_name=sheet_reception)

del sheet_reception

df_receptions.head(2)

,Programme,Bailleur,Nouveau code,Code,Désignation NPSP,Date de réception effective,Date effective de dépotage et/ou expertise,Quantité réceptionnée,Date d'entrée en machine


## 📌6. Importation de la feuille `PPI`

In [13]:
sheet_ppi = compute_indicators.utils.check_if_sheet_name_in_file("PPI", sheetnames)

assert sheet_ppi is not None, print(
    f"La feuille `PPI` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_ppi = pd.read_excel(fp_suivi_stock, sheet_name=sheet_ppi, skiprows=2)

del sheet_ppi

df_ppi.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité
0,3050015,AMODIAQUINE/ARTESUNATE 25 / 67.5 mg ENFANT (2 ...,BTE,EAG523001B,2025-01-31,25
1,3050016,AMODIAQUINE/ARTESUNATE 50 / 135 mg ENFANT (1 -...,BTE,EAC723001A,2025-01-31,18


## 📌7. Importation de la feuille  `Prélèvement`

In [14]:
sheet_prelev = compute_indicators.utils.check_if_sheet_name_in_file(
    "Prelèvement CQ", sheetnames
)

assert sheet_prelev is not None, print(
    f"La feuille `Prelèvement CQ` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_prelevement = pd.read_excel(fp_suivi_stock, sheet_name=sheet_prelev, skiprows=2)

del sheet_prelev

df_prelevement.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité


## 📌8. Importation de la feuille `Plan d'approvisionnement`

In [15]:
import locale

locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

sheet_approv = compute_indicators.utils.check_if_sheet_name_in_file(
    "Plan d'appro",
    sheetnames,
)

assert sheet_approv is not None, print(
    f"La feuille `Plan d'appro` n'est pas dans la liste {sheetnames} du classeur excel"
)

interface = OpenpyxlInterface(wb=src_wb, use_cache=True)
interface.clear_cache()
data_list = []
for row in src_wb[sheet_approv].iter_rows(min_row=0, max_col=19):
    data = []
    for cell in row:
        if cell.column_letter == "P":
            continue
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_approv)
            data.append(result)
        else:
            data.append(cell.value)
    data_list.append(data)

df_plan_approv = pd.DataFrame(data_list[1:], columns=data_list[0])

# df_plan_approv = pd.read_excel(fp_suivi_stock, sheet_name=sheet_approv, engine="openpyxl")
df_plan_approv.columns = df_plan_approv.columns.str.strip()
df_plan_approv["Date updated"] = df_plan_approv["DATE"].dt.strftime("%B-%Y")
df_plan_approv.head(3)

,Standard product code,ID de produit QAT,Produits,ID de l`envoi QAT,Centrale d'achat,Source Financement,Status,Quantite,Facteur de conversion de QAT vers SAGE,Quantité harmonisée (SAGE),DATE,Cout des Produits,Couts du fret,Couts totaux,Acronym,Received?,Coût unitaire moyen (en dollar),Coût unitaire harmonisé,Date updated
0,4030209.0,3340,Giemsa Stain Solution 500 mL,160768,UNICEF,GFATM,Expédié,1071.0,1,1071.0,2025-05-31,11416.86,1369.93,12786.79,GIEMSA (500),0,10.660085,10.660085,mai-2025
1,4030224.0,3651,Immersion Oil 250 mL,160773,UNICEF,GFATM,Reçu,205.0,1,205.0,2025-04-10,3995.45,479.50,4474.95,HUILE A IMMERSION FL/250 ML FLACON,1,19.490000,19.490000,avril-2025
2,4030247.0,4498,Microscope Slide Glass 25 x 75 mm Plain 50...,160778,UNICEF,GFATM,Expédié,18700.0,1,18700.0,2025-05-30,23375.00,2805.00,26180.00,LAME PORTE OBJET BTE/50 BOITE,0,2.727581,2.727581,mai-2025


## 📌9. Importation de la feuille `Statut Produits`

In [16]:
sheet_statut_prod = compute_indicators.utils.check_if_sheet_name_in_file(
    "Statut Produits", sheetnames
)

assert sheet_statut_prod is not None, print(
    f"La feuille `Statut Produits` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_statut_prod = pd.read_excel(fp_suivi_stock, sheet_name=sheet_statut_prod, skiprows=1)
df_statut_prod["programme"] = programme
df_statut_prod.head()

,CODE,Ancien code,CATEGORIE,DESIGNATION DU PRODUIT,Type,Unité niv Central,Unité niv Périphérique,Facteur de conversion \n(De la centrale à la périphérie),Statut Produit,programme


## 📌10. Importation de la feuille `Annexe 1 - Consolidation`

In [17]:
sheet_annexe_1 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Annexe 1 - Consolidation", sheetnames
)

assert sheet_annexe_1 is not None, print(
    f"La feuille `Annexe 1 - Consolidation` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_etat_stock = pd.read_excel(
    fp_suivi_stock, sheet_name=sheet_annexe_1, skiprows=2, usecols="A:T", engine="openpyxl"
).dropna(how="all")

COLUMN_MAPPING = {
    "Stock Théorique fin": "stock_theorique_mois_precedent",
}
df_etat_stock.rename(
    columns={"CODE": "code_produit"},
    inplace=True,
)

df_etat_stock.rename(
    columns=lambda col: next(
        (v for k, v in COLUMN_MAPPING.items() if re.search(k, col, re.I)), col
    )
    if not col.endswith("SAGE") and not col.endswith("Final Attendu")
    else col,
    inplace=True,
)

df_etat_stock.head(3)

,code_produit,Ancien code,CATEGORIE,DESIGNATION DU PRODUIT,Type,Unité niv Central,Unité niv Périphérique,Facteur de conversion \n(De la centrale à la périphérie),stock_theorique_mois_precedent,Distribution effectuée,Quantité reçue entrée en stock,Quantité de PPI,Quantité prélévée en Contrôle Qualité (CQ),Ajustement de stock,Stock Théorique Final SAGE,Stock Théorique Final Attendu,ECARTS,Justification des écarts,Diligences,Dilig. Choisie
1,3010049.0,AY13071,Médicaments,PARACETAMOL 100 mg comp. bte / 100,Non traceur,BOITE/100,COMPRIME,100.0,95338.0,2835.0,0.0,0.0,0.0,NaN,92397.0,92503.0,-106.0,NaN,NaN,NaN
2,3010062.0,AY13115,Médicaments,PARACETAMOL 250 mg comp bte / 100,Non traceur,BOITE/100,COMPRIME,100.0,43217.0,3484.0,0.0,0.0,0.0,NaN,39451.0,39733.0,-282.0,NaN,NaN,NaN
3,3050015.0,AY02027,Médicaments,"AMODIAQUINE/ARTESUNATE 25 / 67,5 mg ENFANT (0 ...",Traceur,BOITE/25,PLAQUETTE,25.0,736.0,255.0,0.0,25.0,0.0,NaN,501.0,456.0,45.0,NaN,NaN,NaN


# IV. Mise à jour des informations

## 1. Mise à jour des informations sur le produit en se basant sur la feuille `Statut Produits`

À l’issue des réunions mensuelles, des ajustements peuvent être effectués sur les informations de produits.
Ces ajustements incluent :

 * l’ajout de nouveaux produits,
 * la suppression de produits existants,
 * la modification des attributs ou métadonnées associées à certains produits.

La fonction suivante a pour objectif de journaliser ces opérations afin d'assurer une traçabilité complète des modifications apportées.

In [18]:
process_statut_prod(df_statut_prod, schema_name, stock_sync_manager)

## 2. Mise à jour des informations Produits en se basant sur la feuille `Plan d'appro`

L'idée est de pouvoir faire des ajustements directement des informations sur les produits en se basant sur la feuille `Plan d'appro` les informations concernées sont:

* La désignation acronym des produits;
* Le facteur de conversion de QAT vers SAGE;
* Le coût unitaire moyen en dollar qui résulte d'un calcul analytique sur une périodicité des extractions QAT.

In [19]:
df_pa = df_plan_approv.rename(
    columns={
        "Acronym": "acronym",
        "Coût unitaire moyen (en dollar)": "cout_unitaire_moyen_qat",
        "Facteur de conversion de QAT vers SAGE": "facteur_de_conversion_qat_sage",
    }
)

In [20]:
%%script false --no-raise-error
stock_sync_manager.synchronize_product_metadata(df_pa, programme)

In [21]:
del df_pa

## 2. Recherche des modifications sur `Annexe 1 - Consolidation`

In [22]:
# %%script false --no-raise-error
interface = OpenpyxlInterface(wb=src_wb, use_cache=True)
interface.clear_cache()
data_list = []
for row in src_wb[sheet_annexe_1].iter_rows(min_row=5, max_col=20):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    data_list.append(data)

In [23]:
# %%script false --no-raise-error
if data_list:
    df_etat_stock = pd.DataFrame(data_list, columns=df_etat_stock.columns)

    df_etat_stock.fillna(np.nan, inplace=True)

    del data_list, data, result

df_etat_stock.head(3)

,code_produit,Ancien code,CATEGORIE,DESIGNATION DU PRODUIT,Type,Unité niv Central,Unité niv Périphérique,Facteur de conversion \n(De la centrale à la périphérie),stock_theorique_mois_precedent,Distribution effectuée,Quantité reçue entrée en stock,Quantité de PPI,Quantité prélévée en Contrôle Qualité (CQ),Ajustement de stock,Stock Théorique Final SAGE,Stock Théorique Final Attendu,ECARTS,Justification des écarts,Diligences,Dilig. Choisie
0,3010049,AY13071,Médicaments,PARACETAMOL 100 mg comp. bte / 100,Non traceur,BOITE/100,COMPRIME,100,95338,2835,0,0,0,NaN,92397,92503,-106,NaN,NaN,NaN
1,3010062,AY13115,Médicaments,PARACETAMOL 250 mg comp bte / 100,Non traceur,BOITE/100,COMPRIME,100,43217,3484,0,0,0,NaN,39451,39733,-282,NaN,NaN,NaN
2,3050015,AY02027,Médicaments,"AMODIAQUINE/ARTESUNATE 25 / 67,5 mg ENFANT (0 ...",Traceur,BOITE/25,PLAQUETTE,25,736,255,0,25,0,NaN,501,456,45,NaN,NaN,NaN


## 3. Recherche des modifications sur les `DMM`

In [24]:
# %%script false --no-raise-error
# DMM échélonnée par mois
interface.clear_cache()
data_list = []
for start, row in enumerate(src_wb[sheet_annexe_1].iter_rows(
    min_row=4,
    min_col=pyxl.utils.column_index_from_string("V"),
    max_col=pyxl.utils.column_index_from_string("BE"),
), start=1):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    if start == 1:
        new_data = []
        unnamed_counter = 22 # Colonne de début des DMM
        for value in data:
            if value is None:
                new_data.append(f"Unnamed: {unnamed_counter}")
                unnamed_counter += 23
            else:
                new_data.append(value)
        data = new_data
    data_list.append(data)
del new_data

In [25]:
# %%script false --no-raise-error
df_dmm = pd.concat(
    [df_etat_stock[["code_produit"]], pd.DataFrame(data_list[1:], columns=data_list[0])],
    axis=1,
)

In [26]:
# %%script false --no-raise-error
df_stock_track_dmm = df_dmm[
    [col for col in df_dmm.columns if "Unnamed" not in str(col)]
]

df_stock_track_dmm = pd.melt(
    df_stock_track_dmm, id_vars="code_produit", var_name="date_report", value_name="dmm"
)  # .drop_duplicates()

df_stock_track_dmm["date_report"] = (
    df_stock_track_dmm["date_report"]
    .apply(lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d"))
    .astype("<M8[ns]")
)

In [27]:
# %%script false --no-raise-error
# Extraction des informations pour le mois courant
interface.clear_cache()
data_list = []
for row in src_wb[sheet_annexe_1].iter_rows(
    min_row=3,
    min_col=pyxl.utils.column_index_from_string("BG"),
    max_col=pyxl.utils.column_index_from_string("BJ"),
):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    data_list.append(data)

data_list = [row for row in data_list if any(row)]

In [28]:
# %%script false --no-raise-error
df_dmm_curent_month = pd.concat(
    [
        df_etat_stock[["code_produit"]],
        pd.DataFrame(data_list[1:], columns=data_list[0]),
    ],
    axis=1,
).dropna(how="all")

df_dmm_curent_month["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")
df_dmm_curent_month.columns = df_dmm_curent_month.columns.str.replace("\n", " ")
df_dmm_curent_month.rename(
    columns={
        "Nbre de mois de considérés": "nbre_mois_consideres",
        "Distributions enregistrées sur les mois de considérés": "distributions_mois_consideres",
        "DMM Calculée  (à valider pour ce mois)": "dmm_calculee",
        "COMMENTAIRE": "commentaire",
    },
    inplace=True,
)

df_dmm_curent_month.head(2)

,code_produit,nbre_mois_consideres,distributions_mois_consideres,dmm_calculee,commentaire,date_report
0,3010049,6,30020,5003.333333,None,2025-03-01
1,3010062,6,35547,5924.500000,None,2025-03-01


In [29]:
# %%script false --no-raise-error
assert (
    df_stock_track_dmm.merge(
        df_dmm_curent_month, how="left", on=["code_produit", "date_report"]
    ).shape[0]
    == df_stock_track_dmm.shape[0]
)

df_stock_track_dmm = df_stock_track_dmm.merge(
    df_dmm_curent_month, how="left", on=["code_produit", "date_report"]
)

df_stock_track_dmm = df_stock_track_dmm.loc[
    df_stock_track_dmm["date_report"] == date_report
]

In [30]:
# %%script false --no-raise-error
cols_df_dmm = df_dmm.columns.to_list()

mapping = {
    col: cols_df_dmm[i - 1]
    for i, col in enumerate(cols_df_dmm)
    if "Unnamed" in str(col) and i > 0
}


df = pd.melt(
    df_dmm,
    id_vars=["code_produit"],
    value_vars=[col for col in df_dmm.columns if not pd.isna(col)],
    var_name="date_report",
)

df["date_report"] = df["date_report"].replace(mapping)  # .map(mapping)

df_stock_track_dmm_histo = (
    df.loc[df["value"] == "X"]
    .drop(columns="value")
    .merge(df.loc[df["value"] != "X"], on=["code_produit", "date_report"])
    .rename(columns={"date_report": "date_report_prev"})
    .sort_values(
        ["code_produit", "date_report_prev"], ascending=[True, True], ignore_index=True
    )
)

df_stock_track_dmm_histo = df_stock_track_dmm_histo.rename(columns={"value":"dmm"})

df_stock_track_dmm_histo["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")

del df, mapping

df_stock_track_dmm_histo.head()

,code_produit,date_report_prev,dmm,date_report
0,3010049,2024-09-01,4633,2025-03-01
1,3010049,2024-10-01,4181,2025-03-01
2,3010049,2024-11-01,8472,2025-03-01
3,3010049,2025-01-01,3915,2025-03-01
4,3010049,2025-02-01,5984,2025-03-01


## 4. Recherche des modifications sur les `CMM`

In [32]:
# %%script false --no-raise-error
eomonth = (pd.to_datetime(date_report) + pd.offsets.MonthEnd(0)).strftime("%Y-%m-%d")

df_stock_prog_nat = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK_PROGRAMME.format(eomonth=eomonth, programme=programme)
)

df_stock_prog_nat["Code_produit"] = df_stock_prog_nat["Code_produit"].astype(int)

df_stock_prog_nat.head(3)

,Programme,MSD,STATUT,CONSO,SDU,CMM,dispo_globale,dispo_globale_cible,dispo_traceur,dispo_traceur_cible,date_report,id_produit_fk,Code_region,region_order,statut_pourcentage,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog
0,PNLP,"1,0",SOUS-STOCK,31789,62881,63895.0,0.018922,0.023611,0.026342,0.026389,2025-03-31,17,NAT,None,0.027778,17,3050061,ARTEMETHER/LUMEFANTRINE 20 / 120 mg (0 - 3ANS)...,PLAQUETTE,Produit non traceur,PRODUITS PNLP,PNLP-1
1,PNLP,"0,7",SOUS-STOCK,2475,12450,16819.5,0.018922,0.023611,0.026342,0.026389,2025-03-31,4882,NAT,None,0.027778,4882,3010103,AMODIAQUINE 75 MG + SULFADOXINE/PYRIMETHAMINE ...,COMPRIME,Produit non traceur,MEDICAMENTS,PNLP-1
2,PNLP,"4,7",SURSTOCK,28635,154550,32958.5,0.018922,0.023611,0.026342,0.026389,2025-03-31,5310,NAT,None,0.027778,5310,4151200,MILDA Dual AI BALLE/50 BALLE -,MOUSTIQUAIRE,Produit non traceur,MEDICAMENTS,PNLP-1


In [33]:
# %%script false --no-raise-error
header_row = list(
    src_wb[sheet_annexe_1].iter_rows(
        min_row=4,
        max_row=4,
        min_col=pyxl.utils.column_index_from_string("BL"),
        max_col=pyxl.utils.column_index_from_string("CJ"),
    )
)[0]

dico_cols = {}
for cell in header_row:
    if not isinstance(cell, pyxl.cell.MergedCell):
        dico_cols[str(cell.value)[:10]] = cell.column

In [34]:
# %%script false --no-raise-error
# Mise à jour des valeur de cellule car la formule ArrondiSup n'est pas prise en charge dans la version actuelle du package utilisée

for row in src_wb[sheet_annexe_1].iter_rows(
    min_row=5, min_col=dico_cols[date_report], max_col=dico_cols[date_report]
):
    for cell in row:
        if gstf.utils.has_formula(cell):
            code_produit, facteur_conversion = (
                src_wb[sheet_annexe_1].cell(cell.row, 1).value,
                src_wb[sheet_annexe_1].cell(cell.row, 8).value,
            )
            df = df_stock_prog_nat.loc[
                df_stock_prog_nat["Code_produit"] == int(code_produit)
            ]
            if not df.empty:
                value = (
                    math.ceil(df.CONSO.sum() / int(facteur_conversion))
                    if not pd.isna(facteur_conversion) and facteur_conversion != 0
                    else 0
                )
                cell = src_wb[sheet_annexe_1].cell(
                    row=cell.row, column=dico_cols[date_report], value=value
                )
            else:
                cell = src_wb[sheet_annexe_1].cell(
                    row=cell.row, column=dico_cols[date_report], value=0
                )

del df

In [35]:
# %%script false --no-raise-error
# CMM échélonnée par mois
interface.clear_cache()
data_list = []
for start, row in enumerate(
    src_wb[sheet_annexe_1].iter_rows(
        min_row=4,
        min_col=pyxl.utils.column_index_from_string("BL"),
        max_col=pyxl.utils.column_index_from_string("CJ"),
    ),
    start=1,
):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    if start == 1:
        new_data = []
        unnamed_counter = 65  # Colonne de début des CMM
        for value in data:
            if value is None:
                new_data.append(f"Unnamed: {unnamed_counter}")
                unnamed_counter += 2
            else:
                new_data.append(value)
        data = new_data
    data_list.append(data)
del new_data

In [36]:
# %%script false --no-raise-error
df_cmm = pd.concat(
    [df_etat_stock[["code_produit"]], pd.DataFrame(data_list[1:], columns=data_list[0])],
    axis=1,
)

In [37]:
# %%script false --no-raise-error
df_stock_track_cmm = df_cmm[
    [col for col in df_cmm.columns if "Unnamed" not in str(col)]
]

df_stock_track_cmm = pd.melt(
    df_stock_track_cmm, id_vars="code_produit", var_name="date_report", value_name="cmm"
)  # .drop_duplicates()

df_stock_track_cmm["date_report"] = (
    df_stock_track_cmm["date_report"]
    .apply(lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d"))
    .astype("<M8[ns]")
)

In [38]:
# %%script false --no-raise-error
# Extraction des informations pour le mois courant
interface.clear_cache()
data_list = []
for row in src_wb[sheet_annexe_1].iter_rows(
    min_row=3,
    min_col=pyxl.utils.column_index_from_string("CW"),
    max_col=pyxl.utils.column_index_from_string("CZ"),
):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    data_list.append(data)
    
data_list = [row for row in data_list if any(row)]

In [39]:
# %%script false --no-raise-error
df_cmm_currenth_month = pd.concat(
    [df_etat_stock[["code_produit"]], pd.DataFrame(data_list[1:], columns=data_list[0])],
    axis=1,
).dropna(how="all")

df_cmm_currenth_month["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")
df_cmm_currenth_month.columns = df_cmm_currenth_month.columns.str.replace("\n", " ")
df_cmm_currenth_month.rename(
    columns={
        "Nbre de mois de considérés": "nbre_mois_consideres",
        "Consommations enregistrées sur les mois de considérés": "conso_mois_consideres",
        "CMM Calculée en fin du mois": "cmm_calculee",
        "COMMENTAIRE": "commentaire",
    },
    inplace=True,
)

df_cmm_currenth_month.head(2)

,code_produit,nbre_mois_consideres,conso_mois_consideres,cmm_calculee,commentaire,date_report
0,3010049,6,27932,4655.333333,None,2025-03-01
1,3010062,6,33836,5639.333333,None,2025-03-01


In [40]:
# %%script false --no-raise-error
assert (
    df_stock_track_cmm.merge(
        df_cmm_currenth_month, how="left", on=["code_produit", "date_report"]
    ).shape[0]
    == df_stock_track_cmm.shape[0]
)

df_stock_track_cmm = df_stock_track_cmm.merge(
    df_cmm_currenth_month, how="left", on=["code_produit", "date_report"]
)

df_stock_track_cmm = df_stock_track_cmm.loc[
    df_stock_track_cmm["date_report"] == date_report
]

In [41]:
# %%script false --no-raise-error
# CMM histo
cols_df_cmm = df_cmm.columns.to_list()

mapping = {
    col: cols_df_cmm[i - 1]
    for i, col in enumerate(cols_df_cmm)
    if "Unnamed" in str(col) and i > 0
}


df = pd.melt(
    df_cmm,
    id_vars=["code_produit"],
    value_vars=[col for col in df_cmm.columns if not pd.isna(col)],
    var_name="date_report",
)

df["date_report"] = df["date_report"].replace(mapping)  # .map(mapping)

df = df.loc[df.value.notna()]

df_stock_track_cmm_histo = (
    df.loc[df["value"] == "X"]
    .drop(columns="value")
    .merge(df.loc[df["value"] != "X"], on=["code_produit", "date_report"])
    .rename(columns={"date_report": "date_report_prev"})
    .sort_values(
        ["code_produit", "date_report_prev"], ascending=[True, True], ignore_index=True
    )
)
df_stock_track_cmm_histo = df_stock_track_cmm_histo.rename(columns={"value": "cmm"})
df_stock_track_cmm_histo["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")

del df, mapping

df_stock_track_cmm_histo.head()

,code_produit,date_report_prev,cmm,date_report
0,3010049,2024-10-01,4616,2025-03-01
1,3010049,2024-11-01,4812,2025-03-01
2,3010049,2024-12-01,4700,2025-03-01
3,3010049,2025-01-01,4001,2025-03-01
4,3010049,2025-02-01,4409,2025-03-01


## 3. Recherche des modifications sur `Annexe 2 - Suivi des Stocks`

In [44]:
df_plan_approv["code_and_date_concate"] = df_plan_approv.apply(
    lambda row: str(int(row["Standard product code"]))
    + "_"
    + row["DATE"].strftime("%Y-%m-%d")
    if not pd.isna(row["Standard product code"])
    else "_" + row["DATE"].strftime("%Y-%m-%d"),
    axis=1,
)


def get_financement_and_delivery_status(code_produit, date_probable_livraison):
    try:
        code_and_date_concate = (
            str(int(code_produit)) + "_" + date_probable_livraison.strftime("%Y-%m-%d")
            if date_probable_livraison
            else np.nan
        )
        index = df_plan_approv.loc[
            df_plan_approv["code_and_date_concate"] == code_and_date_concate
        ].index[0]
        return {
            "financement": df_plan_approv["Source Financement"].iloc[index],
            "status": df_plan_approv["Status"].iloc[index],
        }
    except Exception as e:
        return {"financement": "", "status": ""}

In [45]:
%%script false --no-raise-error
source_sheet_name = "Plan d'appro"
target_sheet_name = "PlanAppro"

if target_sheet_name not in src_wb.sheetnames:
    target_sheet = src_wb.create_sheet(title=target_sheet_name)
else:
    target_sheet = src_wb[target_sheet_name]

if source_sheet_name in src_wb.sheetnames:
    source_sheet = src_wb[source_sheet_name]
    
    for row in source_sheet.iter_rows():
        for cell in row:
            target_cell = target_sheet.cell(
                row=cell.row, 
                column=cell.column,
                value=cell.value
            )
            
            if cell.has_style:
                target_cell.font = cell.font.copy()
                target_cell.border = cell.border.copy()
                target_cell.fill = cell.fill.copy()
                target_cell.number_format = cell.number_format
                target_cell.protection = cell.protection.copy()
                target_cell.alignment = cell.alignment.copy()
    
    for col_letter, col_dim in source_sheet.column_dimensions.items():
        target_sheet.column_dimensions[col_letter].width = col_dim.width
    
    for row_idx, row_dim in source_sheet.row_dimensions.items():
        target_sheet.row_dimensions[row_idx].height = row_dim.height
    
    print(f"Contenu copié de '{source_sheet_name}' vers '{target_sheet_name}'")
else:
    print(f"❌ La feuille '{source_sheet_name}' n'existe pas")

In [46]:
sheet_annexe_2 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Annexe 2 - Suivi des Stocks", sheetnames
)

interface = OpenpyxlInterface(wb=src_wb, use_cache=True)
interface.clear_cache()

dico_cols = {
    "M": "CONSO",
    "N": "SDU",
    "O": "CMM",
}

for start, row in enumerate(
    src_wb[sheet_annexe_2].iter_rows(
        min_row=5,
        max_row=src_wb[sheet_annexe_1].max_row,
        min_col=pyxl.utils.column_index_from_string("M"),
        max_col=pyxl.utils.column_index_from_string("O"),
    ),
    start=5,
):
    code_produit, facteur_conversion = (
        src_wb[sheet_annexe_1].cell(start, 1).value,
        src_wb[sheet_annexe_1].cell(start, 8).value,
    )
    if not code_produit:
        continue

    df = df_stock_prog_nat.loc[df_stock_prog_nat["Code_produit"] == int(code_produit)]

    # Mise à jour des cellules
    for cell in row:
        if gstf.utils.has_formula(cell):
            cell.value = (
                math.ceil(
                    df[dico_cols.get(cell.column_letter)].sum()
                    / int(facteur_conversion)
                )
                if not pd.isna(facteur_conversion) and facteur_conversion != 0
                else 0
            )

    # Mise à jour des éléments de la cellule AJ: "Financement"
    # en supposant que la date probable de livraison n'a pas changé sur base de la date probable de livraison
    cell_financement, cell_date_probable, cell_status, cell_qte_attendue = (
        src_wb[sheet_annexe_2].cell(start, 36),
        src_wb[sheet_annexe_2].cell(start, 37),
        src_wb[sheet_annexe_2].cell(start, 39),
        src_wb[sheet_annexe_2].cell(start, 32),
    )
    if not gstf.utils.has_formula(cell_date_probable):
        value = get_financement_and_delivery_status(
            code_produit, cell_date_probable.value
        )
        if gstf.utils.has_formula(cell_financement):
            cell_financement.value = value["financement"]
        if gstf.utils.has_formula(cell_status):
            cell_status.value = value["status"]

    # il faut également faire une correction des formules de la colonne AF: Quantité attendue
    if gstf.utils.has_formula(cell_qte_attendue):
        cell_qte_attendue.value = df_plan_approv.loc[
            (df_plan_approv["Standard product code"] == code_produit)
            & (df_plan_approv["DATE"] == cell_date_probable.value),
            "Quantité harmonisée (SAGE)",
        ].sum()

In [47]:
%%time
# %%script false --no-raise-error
sheet_annexe_2 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Annexe 2 - Suivi des Stocks", sheetnames
)
interface = OpenpyxlInterface(wb=src_wb, use_cache=True)
interface.clear_cache()
data_list = []
columns_letter = []
for row in src_wb[sheet_annexe_2].iter_rows(
    min_row=5,
    min_col=pyxl.utils.column_index_from_string("I"),
    max_col=pyxl.utils.column_index_from_string("AR"),
):
    data = []
    for cell in row:
        try:
            # ces colonnes ci ne doivent pas subir de modification en règle générale
            if cell.column_letter in {"AA", "AB", "AC", "AD", "AE"}:
                continue
                
            elif gstf.utils.has_formula(cell):
                result = interface.calc_cell(cell.coordinate, sheet_annexe_2)
                data.append(result)
            else:
                data.append(cell.value)
        except Exception as e:
            if cell.column_letter not in columns_letter:
                columns_letter.append(cell.column_letter)
            print(e)
            continue
    data_list.append(data)

CPU times: user 2.47 s, sys: 1.1 ms, total: 2.47 s
Wall time: 2.48 s


In [48]:
columns_name = [    
    "SDU_CENTRAL",
    "DMM_CENTRAL",
    "MSD_CENTRAL",
    "STATUT_CENTRAL",
    
    "CONSO_DECENTRALISE",
    "SDU_DECENTRALISE",
    "CMM_DECENTRALISE",
    "MSD_DECENTRALISE",
    "STATUT_DECENTRALISE",
    "nombre_de_site_en_rupture_annexe_2",
    
    "SDU_NATIONAL",
    "CMM_NATIONAL",
    "MSD_NATIONAL",
    "STATUT_NATIONAL",
    
    "Date de Péremption la plus proche (BRUTE)",
    "Date de Péremption la plus proche",
    "Quantité correspondante",
    "MSD correspondant",
    
    "Qtité attendue Annexe 2",
    "MSD attendu Annexe 2",
    "Qtité réceptionnés non en Stock Annexe 2",
    "MSD reçu Annexe 2",
    "Financement Annexe 2",
    "Date Probable de Livraison Annexe 2",
    "Date Effective de Livraison Annexe 2",
    "Delivery status Annexe 2",
    "Analyse du risque / Commentaires Annexe 2",
    "Diligences au niveau Central Annexe 2",
    "Diligences au niveau périphérique Annexe 2",
    "Responsable Annexe 2",
    "Dilig. Choisie Annexe 2",
]
df_etat_stock_2 = pd.concat(
    [
        df_etat_stock[["code_produit"]],
        pd.DataFrame(data_list, columns=columns_name),
    ],
    axis=1,
)

In [49]:
assert df_etat_stock.merge(df_etat_stock_2, on="code_produit", how="inner").shape[0] == df_etat_stock.shape[0]

df_etat_stock = df_etat_stock.merge(df_etat_stock_2, on="code_produit", how="inner")

df_etat_stock = df_etat_stock.rename(
    columns={
        "Distribution effectuée": "distribution_effectuee",
        "Quantité reçue entrée en stock": "quantite_recue_stock",
        "Quantité de PPI": "quantite_ppi",
        "Quantité prélévée en Contrôle Qualité (CQ)": "quantite_prelevee_cq",
        "Ajustement de stock": "ajustement_stock",
        "Stock Théorique Final SAGE": "stock_theorique_final_sage",
        "Stock Théorique Final Attendu": "stock_theorique_final_attendu",
        "ECARTS": "ecarts",
        "Justification des écarts": "justification_ecarts",
        "Diligences": "diligences",
        "SDU_CENTRAL": "sdu_central_annexe_2",
        "DMM_CENTRAL": "dmm_central_annexe_2",
        "MSD_CENTRAL": "msd_central_annexe_2",
        "STATUT_CENTRAL": "statut_central_annexe_2",
        "CONSO_DECENTRALISE": "conso_decentralise_annexe_2",
        "SDU_DECENTRALISE": "sdu_decentralise_annexe_2",
        "CMM_DECENTRALISE": "cmm_decentralise_annexe_2",
        "MSD_DECENTRALISE": "msd_decentralise_annexe_2",
        "STATUT_DECENTRALISE": "statut_decentralise_annexe_2",
        "SDU_NATIONAL": "sdu_national_annexe_2",
        "CMM_NATIONAL": "cmm_national_annexe_2",
        "MSD_NATIONAL": "msd_national_annexe_2",
        "STATUT_NATIONAL": "statut_national_annexe_2",
        "Date de Péremption la plus proche (BRUTE)": "date_peremption_plus_proche_brute_annexe_2",
        "Date de Péremption la plus proche": "date_peremption_plus_proche_annexe_2",
        "Quantité correspondante": "quantite_correspondante_annexe_2",
        "MSD correspondant": "msd_correspondant_annexe_2",
        "Qtité attendue Annexe 2": "quantite_attendue_annexe_2",
        "MSD attendu Annexe 2": "msd_attendu_annexe_2",
        "Qtité réceptionnés non en Stock Annexe 2": "quantite_non_stockee_annexe_2",
        "MSD reçu Annexe 2": "msd_recu_annexe_2",
        "Financement Annexe 2": "financement_annexe_2",
        "Date Probable de Livraison Annexe 2": "date_probable_livraison_annexe_2",
        "Date Effective de Livraison Annexe 2": "date_effective_livraison_annexe_2",
        "Delivery status Annexe 2": "statut_annexe_2",
        "Analyse du risque / Commentaires Annexe 2": "analyse_risque_commentaires_annexe_2",
        "Diligences au niveau Central Annexe 2": "diligences_central_annexe_2",
        "Diligences au niveau périphérique Annexe 2": "diligences_peripherique_annexe_2",
        "Responsable Annexe 2": "responsable_annexe_2",
        "Dilig. Choisie Annexe 2": "dilig_choisie_annexe_2",
    },
)

df_etat_stock["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")

del df_etat_stock_2

# V.Exportation des données

In [50]:
dim_produit = pd.read_sql(
    f"SELECT * FROM {schema_name}.dim_produit_stock_track where programme='{programme}'",
    stock_sync_manager.civ_engine,
)

## 1. `Etat de Stock`

In [51]:
df_etat_stock = df_etat_stock.merge(
    dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
    on="code_produit",
    how="inner",
).rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})[
    [
        "id_dim_produit_stock_track_fk",
        "stock_theorique_mois_precedent",
        "distribution_effectuee",
        "quantite_recue_stock",
        "quantite_ppi",
        "quantite_prelevee_cq",
        "ajustement_stock",
        "stock_theorique_final_sage",
        "stock_theorique_final_attendu",
        "ecarts",
        "justification_ecarts",
        "diligences",
        "sdu_central_annexe_2",
        "dmm_central_annexe_2",
        "msd_central_annexe_2",
        "statut_central_annexe_2",
        "conso_decentralise_annexe_2",
        "sdu_decentralise_annexe_2",
        "cmm_decentralise_annexe_2",
        "msd_decentralise_annexe_2",
        "statut_decentralise_annexe_2",
        "nombre_de_site_en_rupture_annexe_2",
        "sdu_national_annexe_2",
        "cmm_national_annexe_2",
        "msd_national_annexe_2",
        "statut_national_annexe_2",
        "date_peremption_plus_proche_brute_annexe_2",
        "date_peremption_plus_proche_annexe_2",
        "quantite_correspondante_annexe_2",
        "msd_correspondant_annexe_2",
        "quantite_attendue_annexe_2",
        "msd_attendu_annexe_2",
        "quantite_non_stockee_annexe_2",
        "msd_recu_annexe_2",
        "financement_annexe_2",
        "date_probable_livraison_annexe_2",
        "date_effective_livraison_annexe_2",
        "statut_annexe_2",
        "analyse_risque_commentaires_annexe_2",
        "diligences_central_annexe_2",
        "diligences_peripherique_annexe_2",
        "responsable_annexe_2",
        "dilig_choisie_annexe_2",
        "date_report",
    ]
]

for col in [col for col in df_etat_stock.columns if "msd" in col]:
    df_etat_stock[col] = df_etat_stock[col].apply(
        lambda x: str(round(float(x), 1)).replace(".", ",")
        if not pd.isna(x) and x != "ND" and x != "NA" and x != ""
        else x
    )

In [52]:
df_etat_stock["date_peremption_plus_proche_annexe_2"] = pd.to_numeric(
    df_etat_stock["date_peremption_plus_proche_annexe_2"], errors="coerce", downcast="integer"
)

df_etat_stock["date_peremption_plus_proche_annexe_2"] = pd.to_datetime(
    df_etat_stock["date_peremption_plus_proche_annexe_2"],
    unit="D",
    origin="1899-12-30",
    errors="coerce",
)

In [53]:
for col in [col for col in df_etat_stock.columns if "date_" in col]:
    # df_etat_stock[col] = df_etat_stock[col].astype("datetime64[ns]")
    df_etat_stock[col] = df_etat_stock[col].apply(
        lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
        if len(str(x)) >= 10
        else np.nan
    )

In [54]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

for col in [col for col in df_.columns if "date_" in col]:
    df_[col] = df_[col].astype("datetime64[ns]")

In [55]:
for col in df_etat_stock.columns:
    if col in df_.columns and df_etat_stock[col].dtype != df_[col].dtype:
        try:
            df_etat_stock[col] = df_etat_stock[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_etat_stock[col] = pd.to_numeric(df_etat_stock[col], downcast=df_[col].dtype, errors="coerce")

In [56]:
df_etat_stock = df_etat_stock.replace({pd.NaT: None})

In [81]:
stock_sync_manager.upsert_dataframe(
    df=df_etat_stock,
    table_name="stock_track",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=["id_dim_produit_stock_track_fk", "date_report"],
)

'Upsert de 39 enrégistrements réussie'

## 2. `Distributions`

### 2.1.`Distributions mois courant` 

In [58]:
df_stock_track_dmm = (
    df_stock_track_dmm.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_dmm.head(3)

,date_report,dmm,nbre_mois_consideres,distributions_mois_consideres,dmm_calculee,commentaire,id_dim_produit_stock_track_fk
0,2025-03-01,2835,6.0,30020.0,5003.333333,None,14
1,2025-03-01,3484,6.0,35547.0,5924.500000,None,15
2,2025-03-01,255,6.0,12014.0,2002.333333,None,1


In [60]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_dmm st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

In [61]:
for col in df_stock_track_dmm.columns:
    if col in df_.columns and df_stock_track_dmm[col].dtype != df_[col].dtype:
        try:
            df_stock_track_dmm[col] = df_stock_track_dmm[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_dmm[col] = pd.to_numeric(
                    df_stock_track_dmm[col], downcast=df_[col].dtype, errors="coerce"
                )

In [62]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_dmm,
    table_name="stock_track_dmm",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=["id_dim_produit_stock_track_fk", "date_report"],
)

'Upsert de 39 enrégistrements réussie'

### 2.2.`Distributions hitoriques considérées pour le mois en cours`

In [63]:
df_stock_track_dmm_histo = (
    df_stock_track_dmm_histo.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_dmm_histo.head(3)

,date_report_prev,dmm,date_report,id_dim_produit_stock_track_fk
0,2024-09-01,4633,2025-03-01,14
1,2024-10-01,4181,2025-03-01,14
2,2024-11-01,8472,2025-03-01,14


In [64]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_dmm_histo st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

In [65]:
for col in df_stock_track_dmm_histo.columns:
    if col in df_.columns and df_stock_track_dmm_histo[col].dtype != df_[col].dtype:
        try:
            df_stock_track_dmm_histo[col] = df_stock_track_dmm_histo[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_dmm_histo[col] = pd.to_numeric(
                    df_stock_track_dmm_histo[col], downcast=df_[col].dtype, errors="coerce"
                )

In [66]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_dmm_histo,
    table_name="stock_track_dmm_histo",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=[
        "id_dim_produit_stock_track_fk",
        "date_report",
        "date_report_prev",
    ],
)

'Upsert de 234 enrégistrements réussie'

## 3.`Consommations`

### 3.1. `Consommations mois courant`

In [67]:
df_stock_track_cmm = (
    df_stock_track_cmm.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_cmm.head(3)

,date_report,cmm,nbre_mois_consideres,conso_mois_consideres,cmm_calculee,commentaire,id_dim_produit_stock_track_fk
0,2025-03-01,5394,6.0,27932.0,4655.333333,None,14
1,2025-03-01,6782,6.0,33836.0,5639.333333,None,15
2,2025-03-01,1760,6.0,11978.0,1996.333333,None,1


In [68]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_cmm st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

In [69]:
for col in df_stock_track_cmm.columns:
    if col in df_.columns and df_stock_track_cmm[col].dtype != df_[col].dtype:
        try:
            df_stock_track_cmm[col] = df_stock_track_cmm[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_cmm[col] = pd.to_numeric(
                    df_stock_track_cmm[col], downcast=df_[col].dtype, errors="coerce"
                )

In [70]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_cmm,
    table_name="stock_track_cmm",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=["id_dim_produit_stock_track_fk", "date_report"],
)

'Upsert de 39 enrégistrements réussie'

### 3.2.`Consommations hitoriques considérées pour le mois en cours`

In [71]:
df_stock_track_cmm_histo = (
    df_stock_track_cmm_histo.merge(
        dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]],
        on="code_produit",
        how="inner",
    )
    .drop(columns="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
)

df_stock_track_cmm_histo.head(3)

,date_report_prev,cmm,date_report,id_dim_produit_stock_track_fk
0,2024-10-01,4616,2025-03-01,14
1,2024-11-01,4812,2025-03-01,14
2,2024-12-01,4700,2025-03-01,14


In [72]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_cmm_histo st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

In [129]:
for col in df_stock_track_cmm_histo.columns:
    if col in df_.columns and df_stock_track_cmm_histo[col].dtype != df_[col].dtype:
        try:
            df_stock_track_cmm_histo[col] = df_stock_track_cmm_histo[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_track_cmm_histo[col] = pd.to_numeric(
                    df_stock_track_cmm_histo[col], downcast=df_[col].dtype, errors="coerce"
                )

In [130]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_track_cmm_histo,
    table_name="stock_track_cmm_histo",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=[
        "id_dim_produit_stock_track_fk",
        "date_report",
        "date_report_prev",
    ],
)

'Upsert de 234 enrégistrements réussie'

## 4.`Stock détaillé`

In [132]:
code_col = [col for col in df_stock_detaille.columns if "CODE" in str(col).upper()][0]

df_stock_detaille.rename(
    columns={
        code_col: "code_produit",
        "Désignation": "designation_produit",
        "Emplacement": "emplacement",
        "Date limite de consommation": "date_limite_consommation",
        "Numéro Lot": "numero_lot",
        "Sous lot (Programme)": "sous_lot_programme",
        "Qté \nPhysique": "qte_physique",
        "Qté \nlivrable": "qte_livrable",
        "Unit": "unit",
    },
    inplace=True,
)
df_stock_detaille = df_stock_detaille[
    [
        "code_produit",
        "designation_produit",
        "emplacement",
        "date_limite_consommation",
        "numero_lot",
        "sous_lot_programme",
        "qte_physique",
        "qte_livrable",
        "unit",
    ]
]

df_stock_detaille = df_stock_detaille.loc[
    df_stock_detaille.code_produit.isin(dim_produit.code_produit)
]
df_stock_detaille.head(1)

,code_produit,designation_produit,emplacement,date_limite_consommation,numero_lot,sous_lot_programme,qte_physique,qte_livrable,unit
2,4030247,LAME PORTE OBJET BTE/50,01A01A01,2025-09-01,09323,PNLP,100,0,BTE


In [133]:
df_stock_detaille = (
    df_stock_detaille.groupby(["code_produit", "date_limite_consommation"])[
        ["qte_physique", "qte_livrable"]
    ]
    .sum(min_count=1)
    .reset_index()
)

df_stock_detaille = (
    dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]]
    .merge(df_stock_detaille, on="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
    .drop(columns=["code_produit"])
)

df_stock_detaille["date_report"] = pd.to_datetime(date_report)

In [135]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select st.* 
    from {schema_name}.stock_track_detaille st
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    """
)

In [136]:
for col in df_stock_detaille.columns:
    if col in df_.columns and df_stock_detaille[col].dtype != df_[col].dtype:
        try:
            df_stock_detaille[col] = df_stock_detaille[col].astype(df_[col].dtype)
        except ValueError:
            if df_[col].dtype in ("float64", "int64"):
                df_stock_detaille[col] = pd.to_numeric(
                    df_stock_detaille[col], downcast=df_[col].dtype, errors="coerce"
                )

In [138]:
stock_sync_manager.upsert_dataframe(
    df=df_stock_detaille,
    table_name="stock_track_detaille",
    schema_name=schema_name,
    engine=stock_sync_manager.civ_engine,
    conflict_columns=[
        "id_dim_produit_stock_track_fk",
        "date_limite_consommation",
        "date_report",
    ],
)

'Upsert de 67 enrégistrements réussie'

## 5. `Etat de stock de la NPSP`

Pour cette table ci il faut juste l'exporter vers la base de données

In [ ]:
query = f"""
DELETE FROM {schema_name}.stock_track_npsp
WHERE date_report = '{date_report}' and programme='{programme}';
"""

stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [ ]:
stock_sync_manager.insert_dataframe_to_table(
    df_etat_stock_npsp, table_name="stock_track_npsp", schema_name="suivi_stock"
)

## 6.`Prévision`

In [144]:
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')
df_plan_approv['Date updated'] = df_plan_approv['DATE'].apply(lambda x: x.strftime("%b-%Y"))

In [146]:
df_prevision = compute_indicators.prevision.get_prevision_current_month(
    df_plan_approv.copy(),
    date_report,
    programme,
    stock_sync_manager.civ_engine,
    schema_name="suivi_stock",
)

,id_dim_produit_stock_track_fk,code_produit,ancien_code,stock_central,dmm_central,stock_national,cmm_national,stock_prev_central,stock_prev_national,period_prev,date_report
0,14,3010049,AY13071,92397.0,5003.3335,105553.0,4922.0,18,21,2025-03-01,2025-03-01
1,14,3010049,AY13071,NaN,NaN,NaN,NaN,17,20,2025-04-01,2025-03-01
2,14,3010049,AY13071,NaN,NaN,NaN,NaN,16,19,2025-05-01,2025-03-01


In [ ]:
id_list = list(map(int, df_prevision.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_prevision
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""
stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [ ]:
stock_sync_manager.insert_dataframe_to_table(
    df_prevision.drop(columns=["code_produit", "ancien_code"]), "stock_track_prevision"
)

## 7.`Plan d'approvisionnement`

In [ ]:
query = f"""
DELETE FROM {schema_name}.plan_approv
WHERE date_report = '{date_report}' and programme='{programme}'
"""
stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [152]:
df_pa = df_plan_approv.rename(
    columns={
        "Standard product code": "standard_product_code",
        "ID de produit QAT": "id_produit_qat",
        "Produits": "designation",
        "ID de l`envoi QAT": "id_envoi_qat",
        "Centrale d'achat": "centrale_achat",
        "Source Financement": "source_financement",
        "Status": "status",
        "Quantite": "quantite",
        "Facteur de conversion de QAT vers SAGE": "facteur_conversion_qat_vers_sage",
        "Quantité harmonisée (SAGE)": "quantite_harmonisee_sage",
        "DATE": "date",
        "Cout des Produits": "cout_produits",
        "Couts du fret": "cout_fret",
        "Couts totaux": "cout_total",
    },
)

df_pa = df_pa[
    ["standard_product_code", "id_produit_qat",
     "designation", "id_envoi_qat",
     "centrale_achat", "source_financement",
     "status", "quantite", "facteur_conversion_qat_vers_sage",
     "quantite_harmonisee_sage", "date",
     "cout_produits", "cout_fret", "cout_total",
    ]
]
df_pa["date_report"] = pd.to_datetime(date_report)
df_pa["programme"] = programme
df_pa["standard_product_code"] = df_pa["standard_product_code"].fillna(0)

df_pa.head(2)

,standard_product_code,id_produit_qat,designation,id_envoi_qat,centrale_achat,source_financement,status,quantite,facteur_conversion_qat_vers_sage,quantite_harmonisee_sage,date,cout_produits,cout_fret,cout_total,date_report,programme
0,4030209.0,3340,Giemsa Stain Solution 500 mL,160768,UNICEF,GFATM,Expédié,1071.0,1,1071.0,2025-05-31,11416.86,1369.93,12786.79,2025-03-01,PNLP
1,4030224.0,3651,Immersion Oil 250 mL,160773,UNICEF,GFATM,Reçu,205.0,1,205.0,2025-04-10,3995.45,479.50,4474.95,2025-03-01,PNLP


In [ ]:
stock_sync_manager.insert_dataframe_to_table(df_pa, table_name="plan_approv", schema_name="suivi_stock")

In [ ]:
del df_pa

In [ ]:
# Mise à jour des colonnes qui contienent des valeurs nan, NaN ou None malgré les mises à jours effectuée
from database_operations import queries

stock_sync_manager.civ_cursor.execute(
    queries.QUERY_UPDATE.format(schema_name=schema_name)
)
stock_sync_manager.conn.commit()

# VII. Exportation du Fichier Suivi de Stock vers le `drive` et la `BD`

In [ ]:
download_url = ggdrive.upload_and_return_link(
    fp_suivi_stock.as_posix(),
    date_report_format,
)

In [ ]:
df_download_url = pd.DataFrame(
    data=[
        {
            "programme": programme,
            "download_url": download_url,
            "date_report": date_report,
        }
    ]
)

df_download_url.date_report = df_download_url.date_report.apply(
    lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
)

In [ ]:
stock_sync_manager.synchronize_table_data(
    df_download_url,
    table_name="share_link",
    merge_keys=["programme", "date_report"],
    programme=programme,
)